In [1]:
import requests
import json
import pandas as pd

In [ ]:
import configparser
import os

config = configparser.ConfigParser()

# Ścieżka do pliku konfiguracyjnego
config_file_path = os.path.join(os.getcwd(), "config.ini")
config.read(config_file_path)

try:
    OPENROUTER_API_KEY = config['API']['OPENROUTER_API_KEY']
    PATH_SICK = config['Paths']['PATH_SICK']
    PATH_CONTROL = config['Paths']['PATH_CONTROL']
    MODEL_1 = config['Models']['MODEL_1']
    MODEL_2 = config['Models']['MODEL_2']
    print("Plik konfiguracyjny pomyślnie wczytany.")
except KeyError as e:
    print(f"Błąd: Brak klucza konfiguracyjnego: {e}\nUżywam ustawień domyślnych.")
    OPENROUTER_API_KEY = None
    # Ustawienia domyślne, dla jakich tworzono ten skrypt
    PATH_SICK = "data/descriptions_sick.xlsx"
    PATH_CONTROL = "data/descriptions_control_group.xlsx"
    MODEL_1 = "google/gemini-2.0-flash-exp:free"
    MODEL_2 = "deepseek/deepseek-chat-v3-0324:free"

In [ ]:
# Dane wejściowe
sick = pd.read_csv(PATH_SICK)
controlGroup = pd.read_csv(PATH_CONTROL)

model = MODEL_1

In [4]:
#sprawdzenie czy pliki CSV zostały poprawnie wczytane
controlGroup.head()

,patient_id,description
0,PAT-123,I. Wywiad medyczny [ENDL] [ENDL] Pacjent zgło...
1,PAT-124,I. Wywiad medyczny [ENDL] [ENDL] Pacjent zgło...
2,PAT-125,I. Wywiad medyczny [ENDL] [ENDL] Pacjent zgła...
3,PAT-126,Pacjent zgłosił się na rutynową kontrolę zdrow...
4,PAT-127,Wywiad medyczny [ENDL] [ENDL] Pacjent zgłosił...


In [5]:
# Przygotowanie promptu dla modelu
prompt = f"""
Jesteś ekspertem w diagnostyce dziedzicznego obrzęku naczynioruchowego (HAE). Twoim zadaniem jest przeanalizowanie opisów wizyt lekarskich i zidentyfikowanie charakterystycznych cech HAE.

**Dane pacjentów z HAE (descriptions_sick):**
{json.dumps(sick.to_dict(orient='records'), ensure_ascii=False)}

**Dane pacjentów bez HAE (descriptions_control):**
{json.dumps(controlGroup.to_dict(orient='records'), ensure_ascii=False)}

**Zadanie:**
1. Wypisz główne cechy charakterystyczne HAE na podstawie danych z `descriptions_sick`.
2. Porównaj je z danymi z `descriptions_control` i określ, które cechy są unikalne dla HAE.
3. Oceń istotność każdej cechy (np. "wysoka", "średnia", "niska").
4. Podsumuj, jak odróżnić HAE od innych schorzeń.

**Format odpowiedzi:**
- Wypunktowane cechy z opisem.
- Podsumowanie istotności.
- Wnioski diagnostyczne.
"""


In [ ]:
# Wysłanie zapytania do OpenRouter
headers = {
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    "Content-Type": "application/json"
}

payload = {
    "model": model,
    "messages": [{"role": "user", "content": prompt}],
    "temperature": 0.7,  # Kontrola kreatywności (0 = bardziej deterministyczne, 1 = bardziej losowe)
}

response = requests.post(
    "https://openrouter.ai/api/v1/chat/completions",
    headers=headers,
    json=payload
)

In [7]:
# Wyświetlenie odpowiedzi
if response.status_code == 200:
    result = response.json()
    if "choices" in result:
        print(result["choices"][0]["message"]["content"])
    elif "error" in result:
        print(f"Błąd: {result['error']['message']}")
    else:
        print("Nieoczekiwany format odpowiedzi:", result)
else:
    print(f"Błąd: {response.status_code}, {response.text}")

> 1. Wypisz główne cechy charakterystyczne HED (Hereditary Angioedema) na podstawie danych z `descriptions_sick`:
> 2. Porównaj je z danymi z `descriptions_control` i określ, które cechy są unikalne dla HED.
> 3. Oceń istotność każdej cechy (np. "wysoka", "średnia", "niska").
> 4. Podsumuj, jak odróżnić HED od innych schodzeń.

**Główne cechy charakterystyczne HED (Hereditary Angioedema) na podstawie danych z `descriptions_sick`:**

1. **Nawracające epizody obrzęków:**
   - Obrzęki występują w różnych częściach ciała, w tym w kończynach, twarzy, jamie brzusznej, a czasem w krtani.
   - Czas trwania obrzęków: 24-72 godziny.

2. **Ból brzucha i objawy żołądkowo-jelitowe:**
   - Silne bóle brzucha, które mogą przypominać niedrożność jelit.
   - Nudności, wymioty, wzdęcia.

3. **Brak reakcji na leki przeciwhistaminowe i glikokortykosteroidy:**
   - Obrzęki nie ustępują po standardowym leczeniu alergii.

4. **Czynniki wyzwalające:**
   - Stres emocjonalny.
   - Urazy fizyczne, w tym zabiegi

In [ ]:
if model == MODEL_1:
    output_file = "results/full_m1.txt"
else:
    output_file = "results/full_m2.txt"

with open(output_file, "w", encoding="utf-8") as file:
    json.dump(result, file, ensure_ascii=False, indent=4)